In [2]:
!uv add openai-agents

Resolved 135 packages in 2.64s                                       
Prepared 13 packages in 168ms                                            
░░░░░░░░░░░░░░░░░░░░ [0/13] Installing wheels...                                warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 13 packages in 102ms                              
 + click==8.3.0
 + colorama==0.4.6
 + griffe==1.14.0
 + httpx-sse==0.4.3
 + mcp==1.18.0
 + openai-agents==0.4.1
 + pydantic-settings==2.11.0
 + python-dotenv==1.1.1
 + python-multipart==0.0.20
 + sse-starlette==3.0.2
 + starlette==0.48.0
 + types-requests==2.32.4.20250913
 + uvicorn==0.38.0


In [3]:
import agents

In [4]:
from agents import Agent,function_tool,Runner

In [5]:
import requests

In [6]:
def fetch_url(url):
    jina_reader_base_url = 'https://r.jina.ai/'
    jina_reader_url = jina_reader_base_url + url
    response = requests.get(jina_reader_url)
    return response.content.decode('utf-8') 
    

In [8]:
import requests
from requests.exceptions import RequestException, HTTPError, Timeout, ConnectionError

def fetch_url(url: str) -> str:
    """
    Fetches the content of a URL using the Jina Reader proxy service.

    Args:
        url (str): The original URL to fetch content from.

    Returns:
        str: The decoded content of the response.

    Raises:
        ValueError: If the URL is invalid or the request fails.
    """
    jina_reader_base_url = 'https://r.jina.ai/'
    jina_reader_url = jina_reader_base_url + url

    try:
        response = requests.get(jina_reader_url, timeout=10)
        response.raise_for_status()  # Raises HTTPError for bad status codes (4xx or 5xx)
        return response.content.decode('utf-8')
    except RequestException as e:
        # Catch all network-related errors (e.g., ConnectionError, Timeout, HTTPError)
        print(f"Error fetching URL '{jina_reader_url}': {e}")
        return None
    except UnicodeDecodeError:
        print(f"Error decoding response from '{jina_reader_url}'.")
        return None  


In [9]:
content = fetch_url('https://datatalks.club')

In [11]:
web_agent = Agent(
    name = 'web-agent',
    instructions="you're a helful assistnat",
    model='gpt-4o-mini',
    tools=[function_tool(fetch_url)]
)

In [12]:
runner = Runner()
question = "what is this page about? https://openai.github.io/openai-agents-python/"

In [15]:
result = await runner.run(web_agent, input = question)

/tmp/ipykernel_10707/3773915482.py:1: RuntimeWarning: coroutine 'Runner.run' was never awaited
  result = await runner.run(web_agent, input = question)


In [19]:
print(result.final_output)

The page you referred to is about the **OpenAI Agents SDK**, which is designed to facilitate the development of agentic AI applications using a straightforward, user-friendly package. Here are the key points:

### Overview
- The **OpenAI Agents SDK** allows developers to build applications with AI agents that utilize language models (LLMs).
- It features primitives like **Agents**, **Handoffs**, **Guardrails**, and **Sessions** to manage and maintain conversation histories and delegate tasks.

### Main Features
1. **Agent Loop**: Automatically manages interactions and responses between the agent and tools.
2. **Python-first**: Leverages Python's built-in capabilities for easy orchestration without needing new abstractions.
3. **Handoffs**: Enables agents to delegate specific tasks to other agents.
4. **Guardrails**: Provides validation for inputs and outputs to ensure accuracy.
5. **Sessions**: Manages conversation history without manual intervention.
6. **Function Tools**: Converts Py

In [22]:
items = result.new_items

In [23]:
items[0]

ToolCallItem(agent=Agent(name='web-agent', handoff_description=None, tools=[FunctionTool(name='fetch_url', description='Fetches the content of a URL using the Jina Reader proxy service.', params_json_schema={'properties': {'url': {'description': 'The original URL to fetch content from.', 'title': 'Url', 'type': 'string'}}, 'required': ['url'], 'title': 'fetch_url_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x745c7b68c680>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)], mcp_servers=[], mcp_config={}, instructions="you're a helful assistnat", prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None

In [24]:
items[0].raw_item

ResponseFunctionToolCall(arguments='{"url":"https://openai.github.io/openai-agents-python/"}', call_id='call_l9osn32NF4Cap5YUGpV7lZR1', name='fetch_url', type='function_call', id='fc_0665c6d8cc8182d40068f8ec9ba3d88198a9b90153aa885045', status='completed')

In [26]:
items[-1].raw_item.content[0].text

'The page you referred to is about the **OpenAI Agents SDK**, which is designed to facilitate the development of agentic AI applications using a straightforward, user-friendly package. Here are the key points:\n\n### Overview\n- The **OpenAI Agents SDK** allows developers to build applications with AI agents that utilize language models (LLMs).\n- It features primitives like **Agents**, **Handoffs**, **Guardrails**, and **Sessions** to manage and maintain conversation histories and delegate tasks.\n\n### Main Features\n1. **Agent Loop**: Automatically manages interactions and responses between the agent and tools.\n2. **Python-first**: Leverages Python\'s built-in capabilities for easy orchestration without needing new abstractions.\n3. **Handoffs**: Enables agents to delegate specific tasks to other agents.\n4. **Guardrails**: Provides validation for inputs and outputs to ensure accuracy.\n5. **Sessions**: Manages conversation history without manual intervention.\n6. **Function Tools*

In [27]:
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIAgentsSDKRunner

chat_interface = IPythonChatInterface()

runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=web_agent
)

In [28]:
await runner.run()

You: what are openai agents


You: stop


Chat ended.


In [35]:
from youtube_transcript_api import YouTubeTranscriptApi

def format_timestamp(seconds: float) -> str:
    """Convert seconds to H:MM:SS if > 1 hour, else M:SS"""
    total_seconds = int(seconds)
    hours, remainder = divmod(total_seconds, 3600)
    minutes, secs = divmod(remainder, 60)

    if hours > 0:
        return f"{hours}:{minutes:02}:{secs:02}"
    else:
        return f"{minutes}:{secs:02}"


def make_subtitles(transcript) -> str:
    lines = []

    for entry in transcript:
        ts = format_timestamp(entry.start)
        text = entry.text.replace('\n', ' ')
        lines.append(ts + ' ' + text)

    return '\n'.join(lines)
    
def fetch_transcript_raw(video_id):
    ytt_api = YouTubeTranscriptApi()
    transcript = ytt_api.fetch(video_id)
    return transcript


def fetch_transcript_text(video_id):
    transcript = fetch_transcript_raw(video_id)
    subtitles = make_subtitles(transcript)
    return subtitles

In [44]:
from pathlib import Path

def fetch_transcript_cached(video_id):
    cache_dir = Path("../data_cache/youtube_videos")
    print(cache_dir)
    cache_file = cache_dir / f"{video_id}.txt"

    if cache_file.exists():
        return cache_file.read_text(encoding="utf-8")

    subtitles = fetch_transcript_text(video_id)
    cache_file.write_text(subtitles, encoding="utf-8")

    return subtitles

In [49]:
subtitles = fetch_transcript_cached('2ZOnA19sDpM')
print(subtitles[:500])

../data_cache/youtube_videos
0:00 First of all, thank you very much for uh
0:02 for joining the event. This is the first
0:04 event we ever have. Uh so this is a new
0:07 experience for us as well. So I hope
0:10 everything um goes smoothly
0:13 and this is the first event but not the
0:16 last one. We already have uh a few
0:18 events planned. So the first one, this
0:20 one is about deploying models with
0:22 SageMaker. But u we want to meet every
0:26 Tuesday
0:27 at u 5:00 p.m. uh European time to talk
0:31 about differ


In [50]:
summary_instructions = """
You're a helpful assistant that helps answer user questions
about YouTube videos
"""

In [51]:
def fetch_youtube_transcript(video_id: str) -> str:
    """
    Fetches the transcript of a YouTube video and converts it into a subtitle-formatted string.

    Args:
        video_id (str): The unique YouTube video ID.

    Returns:
        str: The subtitles generated from the video's transcript.
    """
    return fetch_transcript_cached(video_id)

In [52]:
youtube_agent = Agent(
    name='youtube_agent',
    instructions=summary_instructions,
    tools=[function_tool(fetch_youtube_transcript)],
    model='gpt-4o-mini'
)

In [53]:
runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=youtube_agent
)

In [54]:
await runner.run();

You: summarize the video


You: 2ZOnA19sDpM


../data_cache/youtube_videos


You: thanks


You: stop


Chat ended.
